<a href="https://colab.research.google.com/github/MaxMaffio/InterviewQuery/blob/main/Supercell_Data_Scientist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo.png](https://github.com/interviewquery/takehomes/blob/supercell_1/supercell_1/logo.png?raw=1)

# Supercell Data Scientist Pre-Test
---

**Please solve the following tasks.**

## Task 1: How does the revenue trickle?

The database contains three tables: `account`, `account_device` and
`transactions`. `account` contains user profiles, `account_device` their devices and `transactions` contains in-app purchases.

-   How much revenue was produced on 2013-02-01?
-   Are there any users who use both iPads and iPhones?
-   Which country produces the most revenue?
-   What is the iPad/iPhone split in Canada?
-   What proportion of lifetime revenue is generated on the player's
    first week in game?

If you believe the data we've provided is not sufficient for a task,
please outline your concerns in your report with suggestions.

Feel free to use tables and plots.

## Task 2: Visualize this!

Please visualize a single aspect of the data you find important.

-   Why did you choose this particular visualization?
-   What would you improve in your visualization?
-   What would be your conclusions and recommendations to the game team based on this visualization?

## Task 3: Patterns?

Please apply a suitable machine learning technique to the data.

-   Why did you choose this particular technique?
-   What would you improve in your model?
-   What would be your conclusions and recommendations to the game team
    based on this model?



---



In [1]:
!git clone --branch supercell_1 https://github.com/interviewquery/takehomes.git
%cd takehomes/supercell_1
!if [[ $(ls *.zip) ]]; then unzip *.zip; fi
!ls

Cloning into 'takehomes'...
remote: Enumerating objects: 1963, done.
remote: Counting objects: 100% (1963/1963), done.
remote: Compressing objects: 100% (1220/1220), done.
remote: Total 1963 (delta 752), reused 1928 (delta 726), pack-reused 0 (from 0)
Receiving objects: 100% (1963/1963), 297.43 MiB | 10.04 MiB/s, done.
Resolving deltas: 100% (752/752), done.
/content/takehomes/supercell_1
ls: cannot access '*.zip': No such file or directory
account.csv  account_device.csv  logo.png  takehomefile.ipynb  transactions.csv


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import plotly.graph_objects as go
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
os.listdir()

['account_device.csv',
 'transactions.csv',
 'logo.png',
 'account.csv',
 'takehomefile.ipynb']

# READ DATA

In [3]:
# read datasets
df_account = pd.read_csv('account.csv')
df_account_device = pd.read_csv('account_device.csv')
df_transactions = pd.read_csv('transactions.csv')

In [4]:
# account
print(df_account.shape)
print(df_account["account_id"].nunique())
df_account.head()

(20420, 11)
20420


,account_id,created_date,last_login_date,last_login_country,last_login_city,create_country,language,gender,birth_year,age_seconds,session_count
0,0-2786671,2013-01-04 09:28:07,2013-01-04 10:47:40,IT,NaN,IT,EN,0,NaN,1821,2
1,0-2787228,2013-01-04 10:09:03,2013-02-28 17:47:07,QA,NaN,QA,EN,0,NaN,2843,11
2,0-2787785,2013-01-04 10:48:38,2013-01-05 12:48:43,IL,Ramla,IL,EN,0,NaN,553,2
3,0-2788342,2013-01-04 11:26:38,2013-04-03 11:12:09,ES,Terrassa,ES,EN,0,NaN,7895,14
4,0-2788899,2013-01-04 12:02:53,2013-03-09 06:01:59,MN,NaN,MN,EN,0,NaN,25666,41


In [5]:
# account_device
print(df_account_device.shape)
print(df_account_device["account_id"].nunique())
df_account_device.head()

(20420, 2)
20420


,account_id,device
0,0-2786671,"iPod4,1"
1,0-2787228,"iPhone4,1"
2,0-2787785,"iPhone4,1"
3,0-2788342,"iPad2,1"
4,0-2788899,"iPad1,1"


In [6]:
# transactions
print(df_transactions.shape)
print(df_transactions["id"].nunique())
print(df_transactions["account_id"].nunique())
df_transactions.head()

(3788, 6)
3785
1100


,id,account_id,in_game_currency_amoung,created_time,currency_code,cash_amount
0,1603767,0-2845869,1200,2013-01-07 09:31:32,USD,3.99
1,1604990,0-2836366,2500,2013-01-07 13:02:16,USD,6.99
2,1633303,0-2869440,1200,2013-01-10 00:12:07,USD,3.99
3,1653538,0-2869440,500,2013-01-11 00:50:06,USD,1.99
4,1660951,0-2914532,1200,2013-01-11 07:24:42,USD,3.99


As we notice that there are some id transactions duplicated, we check if they are redundant.

In [7]:
df_transactions[df_transactions["id"].duplicated()]

,id,account_id,in_game_currency_amoung,created_time,currency_code,cash_amount
2974,1913991,4-1265456,500,2013-04-04 21:02:24,USD,1.99
3326,573481,5-2136324,500,2013-03-24 22:50:11,USD,1.99
3345,597327,5-2166010,500,2013-03-26 13:43:39,USD,1.99


We can see that there are 3 couple of transactions with same id but different account id. For this reason we decide to keep the records and to add a new column named as id_transaction, in order to have a more reliable and unique id.

In [8]:
# create a new ID
df_transactions["id_transaction"] = df_transactions.index

# TASK 1

## how much revenue was produced on 2013-02-01?

In [9]:
# convert created_date into date
df_transactions['created_date_tr'] = pd.to_datetime(df_transactions['created_time']).dt.date
# sum uop the cash_amount realized on 2013-02-01
cond_1 = df_transactions["created_date_tr"] == pd.to_datetime("2013-02-01", format="%Y-%m-%d").date()
n_outcome = df_transactions.loc[cond_1, "cash_amount"].sum()
# print the outcome
print(f"The revenue on 2013-02-01 was: {n_outcome}")

The revenue on 2013-02-01 was: 159.64000000000004


## are there any users who use both iPads and iPhones?

In [10]:
print(df_account_device.shape)
print(df_account_device["account_id"].nunique())
print(df_account_device[df_account_device["account_id"].duplicated()].shape)

(20420, 2)
20420
(0, 2)


## which country produces the most revenue?

In [11]:
# sum up the cash amount for each account
df_account_cash_amount = df_transactions.groupby(["account_id"], as_index=False).agg({"cash_amount": "sum"})
print(df_account_cash_amount.shape)

# merge with account
df_account_cash_amount = df_account_cash_amount.merge(df_account, on="account_id", how="left")
print(df_account_cash_amount.shape)

# group by country
df_account_cash_amount.groupby(["create_country"], as_index=False).agg({"cash_amount": "sum"}).sort_values(by="cash_amount", ascending=False).head()


(1100, 2)
(1100, 12)


,create_country,cash_amount
59,US,5604.97
21,GB,1288.90
20,FR,1115.55
3,AU,892.71
8,CA,397.96


## what is the iPad/iPhone split in Canada?

In [12]:
# merge account with account_device
df_ac_dev = df_account.merge(df_account_device, on="account_id", how="left")
# clean a name
df_ac_dev["device"] = df_ac_dev["device"].replace("iPhåne2,1", "iPhone")
# create a column to isole the string iPad/iPhone
df_ac_dev['device_name_clean'] = ""
df_ac_dev.loc[df_ac_dev['device'].str.contains('iPad', na=False), 'device_name_clean'] = 'iPad'
df_ac_dev.loc[df_ac_dev['device'].str.contains('iPhone', na=False), 'device_name_clean'] = 'iPhone'
df_ac_dev.loc[df_ac_dev['device'].str.contains('iPod', na=False), 'device_name_clean'] = 'iPhone'

# calcoalte the output
cond_1 = df_ac_dev["create_country"] == "CA"
df_ac_dev_canada = df_ac_dev[cond_1]
df_ac_dev_canada.groupby(["device_name_clean"], as_index=False).agg({"account_id": "count"})

,device_name_clean,account_id
0,iPad,205
1,iPhone,562


## what proportion of lifetime revenue is generated on the player's first week in game?

In [71]:
# Merge transactions with account data to get creation time
df_merged = pd.merge(df_transactions, df_account, on='account_id', how='left')

# Calculate the first week in-game for each player
df_merged['first_week_end'] = min(df_merged['created_date_tr']) + pd.Timedelta(days=7)

# Identify transactions within the first week
df_merged['first_week_transaction'] = np.where(pd.to_datetime(df_merged['created_date_tr']) <= df_merged['first_week_end'], True, False)

# Calculate total revenue and first-week revenue for each player
df_player_revenue = df_merged.groupby('account_id', as_index=False).agg({'cash_amount':"sum"})
df_first_week_revenue = df_merged[df_merged['first_week_transaction']==True].groupby('account_id', as_index=False).agg({'cash_amount':"sum"})
# Combine the revenue data
df_revenue_comparison = df_player_revenue.merge(df_first_week_revenue, on='account_id', how='right', suffixes=('_total', '_first_week'))
df_revenue_comparison.columns = ['account_id', 'total_revenue', 'first_week_revenue']
# Calculate the proportion of lifetime revenue from the first week
df_revenue_comparison['proportion'] = df_revenue_comparison['first_week_revenue'] / df_revenue_comparison['total_revenue']

# Display the result
print(df_revenue_comparison['proportion'].mean())

0.5009304462443743


# TASK 2

Please visualize a single aspect of the data you find important.


In [64]:
df_account_trans_device.head()

,account_id,created_date,last_login_date,last_login_country,last_login_city,create_country,language,gender,birth_year,age_seconds,session_count,id,in_game_currency_amoung,created_time,currency_code,cash_amount,id_transaction,created_date_tr,device,device_name_clean
0,0-2786671,2013-01-04 09:28:07,2013-01-04 10:47:40,IT,NaN,IT,EN,0,NaN,1821,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"iPod4,1",iPhone
1,0-2787228,2013-01-04 10:09:03,2013-02-28 17:47:07,QA,NaN,QA,EN,0,NaN,2843,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"iPhone4,1",iPhone
2,0-2787785,2013-01-04 10:48:38,2013-01-05 12:48:43,IL,Ramla,IL,EN,0,NaN,553,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"iPhone4,1",iPhone
3,0-2788342,2013-01-04 11:26:38,2013-04-03 11:12:09,ES,Terrassa,ES,EN,0,NaN,7895,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"iPad2,1",iPad
4,0-2788899,2013-01-04 12:02:53,2013-03-09 06:01:59,MN,NaN,MN,EN,0,NaN,25666,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"iPad1,1",iPad


In [62]:
# Merge transactions with account data to get creation time
df_account_trans = pd.merge(df_account, df_transactions,  on='account_id', how='left')
df_account_trans_device = pd.merge(df_account_trans, df_account_device,  on='account_id', how='left')
# column date tr
df_account_trans_device['created_date_tr'] = pd.to_datetime(df_account_trans_device['created_time']).dt.date
# create a column to isole the string iPad/iPhone
df_account_trans_device['device_name_clean'] = ""
df_account_trans_device.loc[df_account_trans_device['device'].str.contains('iPad', na=False), 'device_name_clean'] = 'iPad'
df_account_trans_device.loc[df_account_trans_device['device'].str.contains('iPhone', na=False), 'device_name_clean'] = 'iPhone'
df_account_trans_device.loc[df_account_trans_device['device'].str.contains('iPod', na=False), 'device_name_clean'] = 'iPhone'

In [63]:
# Group by country and sum cash amounts
df_country_revenue = df_account_trans_device.groupby(['create_country', 'device_name_clean'], as_index=False).agg({'cash_amount':"sum"})

# Sort by cash amount in descending order
df_country_revenue = df_country_revenue.sort_values('cash_amount', ascending=False)

# select only the firsts more important
df_country_revenue = df_country_revenue.head(10)

# Create the bar chart
fig = go.Figure(data=[go.Bar(
    x=df_country_revenue['create_country'],
    y=df_country_revenue['cash_amount']
)])

fig = go.Figure()
for device in df_country_revenue['device_name_clean'].unique():
    df_device = df_country_revenue[df_country_revenue['device_name_clean'] == device]
    fig.add_trace(go.Bar(
        x=df_device['create_country'],
        y=df_device['cash_amount'],
        name=device,
    ))

fig.update_layout(barmode='stack', title='Revenue by Country and Device', xaxis_title='Country', yaxis_title='Cash Amount')
fig.show()


In [79]:
# Convert 'created_date_tr' to datetime objects for proper time series operations
df_account_trans_device['created_date_tr'] = pd.to_datetime(df_account_trans_device['created_date_tr'])

# Group by year-month, country, and sum cash amount
df_grouped = df_account_trans_device.groupby([
    df_account_trans_device['created_date_tr'].dt.to_period('M'),
    'create_country'
])['cash_amount'].sum().reset_index()

# Rename columns for clarity
df_grouped.columns = ['year_month', 'create_country', 'cash_amount']


# Create the time chart using Plotly
fig = go.Figure()

for country in df_grouped['create_country'].unique():
    df_country = df_grouped[df_grouped['create_country'] == country]
    fig.add_trace(go.Scatter(
        x=df_country['year_month'].astype(str),  # Convert PeriodIndex to string for x-axis
        y=df_country['cash_amount'],
        mode='lines+markers',
        name=country,
    ))

fig.update_layout(
    title='Time Chart of Cash Amount by Country (Summed by Year-Month)',
    xaxis_title='Created Date (Year-Month)',
    yaxis_title='Cash Amount',
    xaxis_tickangle=-45
)

fig.show()